In [1]:
import sys
sys.path.append('..')

import torch
import lightning as L
from torch.utils.data import DataLoader

from dataset import AEDataset
from lightning_model import LitAE

In [2]:
from types import SimpleNamespace
config = {
    'enc_filters': (64, 128, 128, 256),
    'latent_dim': 128,
    'enc_bn': True,
    'dec_features': (256, 256),
    'n_pts': 256,
    'dec_bn': False,
}
config = SimpleNamespace(**config)

In [3]:
root = '..'
dataset_name = 'shapenetcorev2'
# choose split type from 'train', 'test', 'all', 'trainval' and 'val'
# only shapenetcorev2 and shapenetpart dataset support 'trainval' and 'val'

train_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='train')
val_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='val')
test_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
print("datasize:", train_dataset.__len__())
print("datasize:", val_dataset.__len__())
print("datasize:", test_dataset.__len__())

datasize: 35708
datasize: 5158
datasize: 10261


In [4]:
autoencoder = LitAE(config)

In [7]:
for batch in train_loader:
    print(batch)
    print(batch['points'].shape)
    break

{'points': tensor([[[ 8.9962e-02,  1.6842e-01,  4.8156e-02,  ..., -3.7658e-01,
           1.4314e-01,  3.0484e-01],
         [ 6.8876e-02,  5.5784e-01,  5.4834e-02,  ..., -3.4681e-01,
           6.9916e-01, -3.4516e-01],
         [ 2.5466e-01, -7.8273e-02,  1.1071e-01,  ...,  4.2788e-01,
          -8.6583e-02, -4.9160e-01]],

        [[-4.4747e-01,  2.4359e-02,  4.5670e-01,  ..., -5.9050e-01,
          -4.9137e-01, -6.9198e-01],
         [ 4.9071e-01, -1.0630e-01,  9.7348e-02,  ...,  5.6836e-02,
          -1.4527e-01, -6.2099e-01],
         [ 3.4736e-01, -3.7975e-01,  2.0419e-01,  ...,  2.3878e-01,
          -5.6534e-01, -4.4781e-03]],

        [[-6.6747e-02,  3.7337e-02, -1.6324e-01,  ..., -2.2159e-01,
           1.3515e-01, -2.2157e-01],
         [ 1.9849e-01, -1.3094e-01, -1.1861e-01,  ..., -4.0975e-02,
          -3.8280e-02,  8.7529e-03],
         [ 1.8727e-01, -8.0979e-01,  9.1355e-01,  ...,  5.9768e-01,
          -7.9011e-01, -2.0209e-02]],

        ...,

        [[ 1.6449e-02,  

In [5]:
trainer = L.Trainer(max_epochs=1000, gpus=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=val_loader)

/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type    | Params
----------------------------------------
0 | autoencoder | PointAE | 390 K 
----------------------------------------
390 K     Trainable params
0         Non-trainable params
390 K     Total params
1.564     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
